In [63]:
#* project_deliverable_2.ipynb
#*
#* ANLY 555 2023
#* Project <>
#*
#* Due on: 10/04/2023
#* Author(s): Landon Carpenter
#*
#*
#* In accordance with the class policies and Georgetown's
#* Honor Code, I certify that, with the exception of the
#* class resources and those items noted below, I have neither
#* given nor received any assistance on this project other than
#* the TAs, professor, textbook and teammates.
#*

import csv
import nltk
import numpy as np
import matplotlib.pyplot as plt
import os
import wordcloud

#create dataset class
class DataSet:
    """
    Class for managing the dataset
    
    Attribute:
        filename (str): the name of the file to be read in
    """

    #constructor
    def __init__(self, filename, ):
        """
        Initializes the DataSet class
        """
        self.filename = filename
        self.data = None

    #create the framework and stubs for __readFromCSV, __load, clean, and explore
    def __readFromCSV(self, filename, header = True):
        """
        Reads in the data from a CSV file. The data is stored on a column basis similar to a parquet file to more easily account for data types.
        """
        try:
            with open(filename, 'r') as f:
                reader = csv.reader(f)
                data = list(reader)

                if header:
                    header = data[0]
                    data = data[1:]
                else:
                    header = [f"col_{i}" for i in range(len(data[0]))]

                #init the dict to store the data
                columns = {col_name: [] for col_name in header}

                for row in data:
                    for col_name, value in zip(header, row):
                        try:
                            columns[col_name].append(float(value))
                        except:
                            columns[col_name].append(value)
                            #if the value is '' then replace it with np.nan
                            if value == '':
                                columns[col_name][-1] = np.nan
                            else:
                                pass


                #ok now convert to numpy array
                d_type = [(col_name, object if any(isinstance(val, str) for val in columns[col_name]) else float) for col_name in header]
                self.data = np.array(list(zip(*[columns[col_name] for col_name in header])), dtype = d_type)

        except Exception as e:
            print(f"Error reading {filename}: {str(e)}")

        return self.data

                    
                        



    #abstract base class (ABC)
    def __load(self, filename):
        """
        Loads the data from a CSV file
        """
        print(f"Loading {filename}...")

        self.__readFromCSV(filename)

        return self.data


    def getType(self):
        """
        This function will be called later in each of the child classes to determine the type of data
        """

        #using while True to avoid infinite loop that I had earlier
        while True:
            data_type = input("Is this data Time Series, Text, Quantitative, or Qualitative? \nPlease type 'Time', 'Text', 'Quantitative', or 'Qualitative'.")
            #trying to make the prompt a little more forgiving by making the input lowercase and removing whitespace before checking for validity
            norm = data_type.lower().strip()

            #make sure the type is valid
            if norm in ['time', 'text', 'quantitative', 'qualitative']:
                return norm

            #if the type is not valid they will see this message and be prompted to try again
            else:
                print("Please enter a valid data type.")


    def clean(self):
        """
        Cleans the data
        """
        print("Cleaning...")

    def explore(self):
        """
        Explores the data
        """
        print("Exploring...")
        

#use inheritance to create TimeSeriesDataSet class
class TimeSeriesDataSet(DataSet):
    """
    Class for managing the time series dataset
    """

    #constructor
    def __init__(self, filename):
        """
        Initializes the TimeSeriesDataSet class
        """
        super().__init__(filename)
    #override the clean and explore methods from the DataSet class to be specific to the TimeSeriesDataSet class
    def clean(self):
        """
        Cleans the time series data set
        """
        print("Cleaning Time Series Data Set...")
    def explore(self):
        """
        Explores the time series data set
        """
        print("Exploring Time Series Data Set...")


class TextDataSet(DataSet):
    """
    Class for managing the text dataset. 
    """

    def __init__(self, filename):
        """
        Initializes the TextDataSet class
        """
        super().__init__(filename)

    def clean(self):
        """
        Cleans the text data set
        """
        print("Cleaning Text Data Set...")

    def explore(self):
        """
        Explores the text data set
        """
        print("Exploring Text Data Set...")


#use inheritance to create QuantDataSet class
class QuantDataSet(DataSet):
    """
    Class for managing the quantitative dataset
    """

    #constructor
    def __init__(self, filename):
        """
        Initializes the QuantDataSet class
        """
        super().__init__(filename)
        self.filename = filename
        self.data = self._DataSet__load(filename)

    #override the clean and explore methods from the DataSet class to be specific to the QuantDataSet class
    def clean(self, header = True):
        """
        Cleans the quantitative data set
        """
        try: 
            if self.data is None:
                self._DataSet__load(self.filename)

            print("Cleaning Quant Data Set...")

            #iterate by column replacing missing values with the mean
            for col_name in self.data.dtype.names:
                col_data = self.data[col_name]

                #if the data is numeric
                if np.issubdtype(col_data.dtype, np.number):
                    #replace missing values with the mean limited to 2 decimal places
                    col_data[np.isnan(col_data)] = np.round(np.nanmean(col_data), 2)
                    
        except Exception as e:
            print(f"Error cleaning {self.filename}: {str(e)}")

    def explore(self):
        """
        Explores the quantitative data set
        """
        print("Exploring Quant Data Set...")

#use inheritance to create QualDataSet class
class QualDataSet(DataSet):
    """
    Class for managing the qualitative dataset
    """

    #constructor
    def __init__(self, filename):
        """
        Initializes the QualDataSet class
        """
        super().__init__(filename)
        self.filename = filename
        self.data = self._DataSet__load(filename)

    #override the clean and explore methods from the DataSet class to be specific to the QualDataSet class
    def clean(self):
        """ 
        Cleans the qualitative data set
        """
        print("Cleaning Qual Data Set...")

        try:
            if self.data is None:
                self._DataSet__load(self.filename)

            #iterate each column and replace missing values with the median
            for col_name in self.data.dtype.names:
                col_data = self.data[col_name]

                #if the data is numeric
                if np.issubdtype(col_data.dtype, np.number):
                    #replace missing values with the median limited to 2 decimal places
                    col_data[np.isnan(col_data)] = np.round(np.nanmedian(col_data), 2)

                #else if the data is not numeric replace the np.nan with the most common value (mode)
                else:
                    #convert any np.nan to 'nan' so that the mode can be calculated
                    col_data[col_data != col_data] = 'nan'
                    #replace missing values with the mode 
                    mode = max(set(col_data), key = col_data.tolist().count)
                    col_data[col_data == 'nan'] = mode

        except Exception as e:
            print(f"Error cleaning {self.filename}: {str(e)}")
                    
    def explore(self):
        """
        Explores the qualitative data set
        """
        print("Exploring Qual Data Set...")


#create class for the classifier 
class ClassifierAlgotithm:
    """
    Class for managing the classifier algorithm
    """
    def __init__(self):
        """
        Initializes the ClassifierAlgotithm class
        """
        pass

    def train(self):
        """
        Trains the classifier algorithm
        """
        print("Training...")

    def test(self):
        """
        Tests the classifier algorithm
        """
        print("Testing...")

#create class for simpe KNN that inherets from ClassifierAlgotithm
class simpleKNNClassifier(ClassifierAlgotithm):
    """
    Class for managing the simple KNN classifier
    """
    def __init__(self):
        """
        Initializes the simpleKNNClassifier class
        """
        super().__init__()

#create class for kdTree KNN that inherets from ClassifierAlgotithm
class kdTreeKNNClassifier(ClassifierAlgotithm):
    """
    Class for managing the kdTree KNN classifier
    """
    def __init__(self):
        super().__init__()

#create the Experiment class that will run cross validation, get a score given k and, and create a confusion matrix
class Experiment:
    """
    Class for managing the experiment
    """

    def __init__(self):
        """
        Initializes the Experiment class
        """
        pass

    def runCrossVal(self, k):
        """
        Runs k-fold cross validation

        Args:
            k (int): the number of folds to use
        """
        print(f"Running {k}-fold cross validation...")

    def score(self):
        """
        Scores the experiment
        """
        print("Scoring...")

    def __confusionMatrix(self):
        """
        Creates a confusion matrix
        """
        print("Creating confusion matrix...")

# #main function: if the py file is ran directly then run the main function and perform the following (rather than importing the file and using a function, for example)
# if __name__ == "__main__":
#     print("Running main to show framework and stubs...")
#     print("\n")
#     print("---Creating dataset---")
#     dataset = DataSet("my_data.csv")
#     dataset._DataSet__readFromCSV("my_data.csv")
#     dataset._DataSet__load("my_data.csv")
#     dataset.clean()
#     dataset.explore()
#     print("\n")
    
#     print("---Creating TimeSeriesDataSet--- ")
#     ts_dataset = TimeSeriesDataSet("my_data.csv")
#     ts_dataset._DataSet__readFromCSV("my_data.csv")
#     ts_dataset._DataSet__load("my_data.csv")
#     ts_dataset.clean()
#     ts_dataset.explore()
#     print("\n")

#     print("---Creating QuantDataSet---")
#     quant_dataset = QuantDataSet("my_data.csv")
#     quant_dataset._DataSet__readFromCSV("my_data.csv")
#     quant_dataset._DataSet__load("my_data.csv")
#     quant_dataset.clean()
#     quant_dataset.explore()
#     print("\n")

#     print("---Creating QualDataSet---")
#     qual_dataset = QualDataSet("my_data.csv")
#     qual_dataset._DataSet__readFromCSV("my_data.csv")
#     qual_dataset._DataSet__load("my_data.csv")
#     qual_dataset.clean()
#     qual_dataset.explore()
#     print("\n")

#     print("---Creating simple KNN classifiers---")
#     simple_knn = simpleKNNClassifier()
#     simple_knn.train()
#     simple_knn.test()
#     print("\n")

#     print("---Creating kdTree KNN classifiers--- ")
#     kd_tree_knn = kdTreeKNNClassifier()
#     kd_tree_knn.train()
#     kd_tree_knn.test()
#     print("\n")

#     print("---Creating experiment--- ")
#     experiment = Experiment()
#     experiment.runCrossVal(5)
#     experiment.score()
#     experiment._Experiment__confusionMatrix()

In [64]:
#read in "test.csv" as my_test using DataSet and __readFromCSV
test_object = DataSet("Quant_Test.csv")
vars(test_object)

{'filename': 'Quant_Test.csv', 'data': None}

In [65]:
#load the data using __load as quant_test
quant_test = QuantDataSet("Quant_Test.csv")
print(f"---Vars---")
print(vars(quant_test))
#clean the data using clean
quant_test.clean()
print("---Vars---")
print(vars(quant_test))

Loading Quant_Test.csv...
---Vars---
{'filename': 'Quant_Test.csv', 'data': array([('P1', 11., 12., 10., 8., 13., 12., 14., 21., nan, 14., 11., 14., 16., 9.,  9., 9., 3., 21., 0.44, 0.5 , 0.39, 0.28, 0.56, 0.5 , 0.61, 1.  ,  nan, 0.61),
       ('P2',  7.,  6., nan, 2.,  7.,  1.,  6.,  3.,  3.,  3., nan,  2.,  6., 2.,  0., 6., 0., 10., 0.7 , 0.6 , 0.3 ,  nan, 0.7 , 0.1 , 0.6 , 0.3 , 0.3 , 0.3 ),
       ('P3',  7., 11.,  8., 9., 10.,  8.,  7., 13., 12.,  6., 14.,  9., nan, 7., 12., 8., 3., 14., 0.36, 0.73, 0.45, 0.55, 0.64, 0.45, 0.36, 0.91, 0.82,  nan),
       ('P4', nan,  8., 13., 5.,  9.,  6., nan, 13., 13., 11.,  8.,  4.,  5., 4., 15., 7., 2., 19., 0.59, 0.35, 0.65, 0.18, 0.41, 0.24,  nan, 0.65,  nan, 0.53)],
      dtype=[('Product_Code', 'O'), ('W0', '<f8'), ('W1', '<f8'), ('W2', '<f8'), ('W3', '<f8'), ('W4', '<f8'), ('W5', '<f8'), ('W6', '<f8'), ('W7', '<f8'), ('W8', '<f8'), ('W9', '<f8'), ('W10', '<f8'), ('W11', '<f8'), ('W12', '<f8'), ('W13', '<f8'), ('W14', '<f8'), ('W15', '<f8'

In [66]:
qual_test = QualDataSet("Qual_Test.csv")
print(f"---Vars---")
print(vars(qual_test))
#clean the data using clean
qual_test.clean()
print("---Vars---")
print(vars(qual_test))

Loading Qual_Test.csv...
---Vars---
{'filename': 'Qual_Test.csv', 'data': array([(510., '22-24', 'Male', -1., 'France', 'Master’s degree', 'Software Engineer', -1., '1000-9,999 employees', 0.0, 'I do not know', nan, nan, nan, nan, nan, nan, nan, nan, -1., '30,000-39,999', '$0 (USD)', 'Twitter (data science influencers)', nan, nan, 'Kaggle (forums, blog, social media, etc)', nan, nan, nan, 'Blogs (Towards Data Science, Medium, Analytics Vidhya, KDnuggets etc)', 'Journal Publications (traditional publications, preprint journals, etc)', nan, nan, nan, -1., nan, 'Coursera', nan, 'DataCamp', nan, 'Kaggle Courses (i.e. Kaggle Learn)', nan, 'Udemy', nan, nan, nan, nan, -1., 'Basic statistical software (Microsoft Excel, Google Sheets, etc.)',  0., -1., -1., -1., -1., -1., '1-2 years', 'Jupyter (JupyterLab, Jupyter Notebooks, etc) ', ' RStudio ', ' PyCharm ', nan, ' MATLAB ', nan, '  Spyder  ', nan, nan, nan, nan, nan, -1., nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 'None', nan, -1., 'Py

D:\Users\carpenterl\AppData\Roaming\Python\Python311\site-packages\numpy\lib\nanfunctions.py:1217: RuntimeWarning: All-NaN slice encountered
  r, k = function_base._ureduce(a, func=_nanmedian, axis=axis, out=out,
